# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [40]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_query('select * from "disaster_response_tweets"', con=engine)


In [10]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# cats = 
cats_df = df[df.columns.tolist()[4:]].sum().reset_index().rename(columns={'index':'cat_name', 0:'count'}).sort_values(by='count',ascending=False)
cats_df

,cat_name,count
0,related,20282
3,aid_related,10860
28,weather_related,7297
35,direct_report,5075
1,request,4474
18,other_aid,3446
11,food,2923
32,earthquake,2455
30,storm,2443
12,shelter,2314


In [42]:
# text = df.message.tolist()
# tokenized=[]
# for txt in text:
#     tokenized.append(word_tokenize(txt))
print(df.columns.tolist())

['id', 'message', 'original', 'genre', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [43]:
# df['tokenized'] = tokenized

### 2. Write a tokenization function to process your text data

In [44]:
def tokenize(text):
#     remove any punctuation character
    text = re.sub(r"[^a-zA-Z0-9]",' ',text)
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

# tokenize(df.message[424])

In [45]:
X = df['message'].values
y = df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']].values

In [46]:
df[df.columns.tolist()[5:]].values==y

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [47]:
cc = df[df.columns.tolist()[5:]].values
cc==y

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


False

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [48]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
#     ('rfc', RandomForestClassifier())
    ('moc', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
y_test

array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [50]:



# Instantiate transformers and classifier
# vect = CountVectorizer(tokenizer=tokenize)
# tfidf = TfidfTransformer()
# moc = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)


# # Fit and/or transform each to the data

# X_train_counts = vect.fit_transform(X_train)
# X_train_tfidf = tfidf.fit_transform(X_train_counts)
# moc.fit(X_train_tfidf, y_train)

pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

In [65]:
# # Transform test data

# X_test_counts = vect.transform(X_test)
# X_test_tfidf = tfidf.transform(X_test_counts)

# # Predict test labels
# y_pred = moc.predict(X_test_tfidf)
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
#     ('rfc', RandomForestClassifier())
    ('moc', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])
pipeline1.fit(X_train,y_train)
y_pred = pipeline1.predict(X_test)
# print(classification_report(np.hstack(y_test),np.hstack(y_pred)))
print(classification_report(np.hstack(y_test),np.hstack(y_pred), labels=labels, target_names =labels))

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.00      0.00      0.00         0
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0
           aid_related       0.00      0.00      0.00         0
          medical_help       0.00      0.00      0.00         0
      medical_products       0.00      0.00      0.00         0
     search_and_rescue       0.00      0.00      0.00         0
              security       0.00      0.00      0.00         0
              military       0.00      0.00      0.00         0
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00         0
                  food       0.00      0.00      0.00         0
               shelter       0.00      0.00      0.00         0
              clothing       0.00      0.00      0.00         0
                 money       0.00      

In [52]:
accuracy = (y_pred == y_test).mean()
accuracy

0.9441350489946767

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [53]:
from sklearn.utils.multiclass import type_of_target

In [54]:
type_of_target(y_pred)

'multiclass-multioutput'

In [64]:
labels = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']
print(classification_report(np.hstack(y_test),np.hstack(y_pred), labels=labels, target_names =labels))

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.00      0.00      0.00         0
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0
           aid_related       0.00      0.00      0.00         0
          medical_help       0.00      0.00      0.00         0
      medical_products       0.00      0.00      0.00         0
     search_and_rescue       0.00      0.00      0.00         0
              security       0.00      0.00      0.00         0
              military       0.00      0.00      0.00         0
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00         0
                  food       0.00      0.00      0.00         0
               shelter       0.00      0.00      0.00         0
              clothing       0.00      0.00      0.00         0
                 money       0.00      

In [60]:
# pipeline1.get_params()

### 6. Improve your model
Use grid search to find better parameters. 

In [63]:
parameters ={
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (2, 2)),
#     'vect__stop_words': (None, 'english'),
    'tfidf__norm': ('l1', 'l2'),
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'tfidf__use_idf': (True, False),
 
}

cv = GridSearchCV(pipeline, parameters,cv=5,n_jobs=-1, verbose=1)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(classification_report(np.hstack(y_test),np.hstack(y_pred)))

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier

pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('dtc', ExtraTreeClassifier())
#     ('moc', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline2.fit(X_train,y_train)
y_pred = pipeline2.predict(X_test)
print(classification_report(np.hstack(y_test),np.hstack(y_pred)))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95    215056
           1       0.52      0.44      0.48     20844
           2       0.26      0.36      0.30        44

   micro avg       0.91      0.91      0.91    235944
   macro avg       0.57      0.59      0.58    235944
weighted avg       0.91      0.91      0.91    235944



In [37]:
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier

pipeline3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('dtc', DecisionTreeClassifier())
#     ('moc', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline3.fit(X_train,y_train)
y_pred = pipeline3.predict(X_test)
print(classification_report(np.hstack(y_test),np.hstack(y_pred)))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96    215056
           1       0.60      0.51      0.55     20844
           2       0.30      0.45      0.36        44

   micro avg       0.93      0.93      0.93    235944
   macro avg       0.62      0.64      0.63    235944
weighted avg       0.92      0.93      0.92    235944



### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.